<a href="https://colab.research.google.com/github/stefangraser/STS_SBERT_Centric2025/blob/main/STS_SBERT_Centric2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch

# Lade die Excel-Datei mit mehreren Spalten
df = pd.read_excel("file.xlsx")

# Stelle sicher, dass die Spalte "Guideline" vorhanden ist
if "Guideline" not in df.columns:
    raise ValueError()

# Modell laden
model = SentenceTransformer('all-mpnet-base-v2')

# ARcis Topics mit Definition + Items
topics = {
    "3D modeling - Visual Realism and Appearance of Objects": [
        "When you create 3D objects, create them to be life size. Full-size objects are easier to drop straight into your experience",
        "All objects should face the same direction. Use a right-handed coordinate space, where +Y is up, +X is right, and -Z points forward from the origin",
        "When you model an object for your scene, make sure to place it on the ground plane at the geometric center of the object base",
        "Remember, 3D objects can be viewed from all sides. Use complete objects, and be sure to render all surfaces, even those that a user might not immediately see, like the back surface of a curtain, or the bottom of a couch"
    ],
    "AR Badges / Glyphs": [
        "You can include a button, such as an AR icon, to let users trigger the launch themselves. Send the user gently into your AR environment",
        "Keep badge placement consistent and clear. A badge looks best when displayed in one corner of an object's photo. Always place it in the same corner and make sure it's large enough to be seen clearly (but not so large that it occludes important detail in the photo).",
        "Use badging only when your app contains a mixture of objects that can be viewed in AR and objects that cannot. If all objects in your app can be viewed in AR, then badging is redundant.",
        "Apps that include collections of products or other objects can use badging to identify specific items that can be viewed in AR using ARKit. For example, a department store app might use a badge to mark furniture that people can preview in their home before making a purchase.",
        "Maintain minimum clear space. The minimum amount of clear space required around an AR badge is 10% of the badge's height. Other elements shouldn't infringe on this space and occlude the badge in any way",
        "Use the AR badges as intended and don’t alter them. Use these images exclusively to identify products or other objects that can be viewed in AR using ARKit. Never alter the badges, change their color, use them for other purposes, or use them in conjunction with AR experiences not created with ARKit.",
        "Prefer the AR badge to the glyph-only badge. In general, use the glyph-only badge for constrained spaces that can't accommodate the AR badge. Both badges work well at their default size.",
        "Use the AR glyph as intended. The glyph should be used strictly for initiating an ARKit-based experience. Never alter the glyph (other than adjusting its size and color), use it for other purposes, or use it in conjunction with AR experiences not created using ARKit.",
        "Maintain minimum clear space. The minimum amount of clear space required around an AR glyph is 10% of the glyph's height. Don’t let other elements infringe on this space or occlude the glyph in any way."
    ],
    "Accessibility (hearing)": [
        "Ensure the experience is completable without audio",
        "Offering the option of a mono downmix.",
        "Offering options for communicating information shared audibly, through visual / haptic cues",
        "Optional ‘turn up’ of the universal tell indicators.",
        "Providing separate volume controls for music, effects, and speech."
    ],
    "Accessibility (mobility)": [
        "Where possible, allow for play to be experienced from a bed, or a fixed seated location, or a wheelchair.",
        "Avoid requiring buttons to be held down or provide an alternative control scheme such as toggling buttons rather than holding them down. This is so a person in a wheelchair can have their hands free while moving.",
        "People with reduced range of motion may need small, precise movements to be amplified."
    ],
    "Accessibility (visuals)": [
        "Many individuals have reduced visual acuity. This means that they cannot read small text or discern fine details in content. Consider providing pre-recorded voice over for all text, or a text to speech engine.",
        "Please consider color blind individuals when designing your content.",
        "Some users have photosensitive epilepsy which can be triggered by certain types of visual content. Take care to avoid high contrast flickering content in your content scenes.",
        "To aid these individuals, choose image layouts that are discriminable in grey scale. It is advised to additionally use other cues such as shape, labels and haptics"
    ],
    "Affordance": [
        "Specific manipulation requires precision. We recommend that you use affordance-based manipulation because it provides a high level of granularity. For flexible manipulation, we recommend you use non-affordance manipulation as it allows for instant and playful experiences. (Microsoft p.171)",
        "For example, if we encourage the user to grab an object or a control point with a two finger pinch, the object or the control point should be small. If we want the user to perform five finger grab, the object or the control point should be relatively large. Similar to buttons, a tiny button would limit users to press it with a single finger; a large button would encourage users to press it with their palms. (Microsoft p.171)",
        "Use a simple design for front-facing camera effects. The effect itself should communicate what can be done with it.",
        "Use What They Know: Leverage real-world intuition as you design interactions.",
        "Responsiveness. Maintain low-latency, precise interactions. Use multisensory feedback to reinforce the clarity and discoverability of these interactions.",
        "Responsiveness. Maintain low-latency, precise interactions. Use multisensory feedback to reinforce the clarity and discoverability of these interactions.",
        "Take care not to visually overpower your virtual objects. Above all, they should still look and act like normal objects in your AR world. With a little added touch, you can encourage users to unlock the secrets of those objects.",
        "Make it easy for users to spot interactive objects and then interact with them. Interacting should be intuitive. Use color combinations, glowing outlines, or other visual highlights to let users know. This is especially critical in apps where multiple objects can be selected."
    ],
    "Anchored UI": [
        "Preserve the user’s sense of immersion by attaching all UI elements to a logical, familiar place.",
        "Opt for presenting supporting information or controls in the real world. This not only saves screen space but also links the content directly to the object",
        "VR and AR experiences should typically attach UI elements to the environment, a tracked controller, or the user’s body. “Anchored” UIs provide higher cognitive ease and require less time to learn.",
        "Tightly couple billboarding or tag-along to the display of Progress since the user can easily move their head into empty space and lose context. Your app might look like it has crashed if the user is unable to see anything. (Microsoft p.381)",
        "Interfaces in mixed reality experiences ideally are paired tightly with the digital content they control. As such, free floating 2D menus are often not ideal for interaction and can be difficult for users to comfortably with inside the holographic frame. For experiences that do require interface elements such as menus or text fields, consider using a tag-along method to follow the holographic frame after a short delay. (Microsoft p.115)"
    ],
    "Animations": [
        "Rapid animations that move an object towards or away from your user are more likely to cause discomfort owing to the transition between the near and far field optics. A fast-moving object is also harder to track with the limited field of view. Slower motion works better.",
        "Only use animation to add value to an experience.",
        "Animation should remain interesting when viewed repeatedly. Otherwise, make the motion subtler.",
        "Animating digital content can add personality and delight to spatial computing. Motion defines spatial relationships, implies functionality, and describes intention."
    ],
    "Attention directors": [
        "The easiest way to let the user know they should move in a specific direction is to provide a visual indication. This can be done through a visual effect (for example, a path the user can visually follow toward the next part of the experience) or even as simple directional arrows. Note that any visual indicator should grounded within the user's environment, not 'attached' to the holographic frame or the cursor(Microsoft p.115)",
        "Pin content in space or to real-world objects or use cues like sound or visual affordances to guide users towards content.",
        "Soundfield Audio: Use 3D spatialized sound cues to help users track objects out of view and maintain presence.",
        "Use affordances such as sounds and arrows to guide towards important notifications that are off the screen.",
        "Use single points of focus whenever possible. This lets users follow the action without feeling lost and keeps the action within their field of view.",
        "Consider other ways to add additional senses (audio and haptics) into their experience. This will enhance the experience of sighted users, and potentially extend the accessibility of the effect to partiallysighted users",
        "Spatial sound can provide a powerful way to establish objects in a scene (alerting users of objects entering an experience) or to direct attention to a specific point in space (to move the user's view toward key objects). Using audio directors to guide the user’s attention can be more subtle and less intrusive than visual directors. In some cases, it can be best to start with an audio director, then move on to a visual director if the user does not recognize the cue. Audio directors can also be paired with visual directors for added emphasis.(Microsoft p.115)",
        "With large distances between objects, consider drawing lines on the floor to lead users to their goal.",
        "Experiences with a vignette can color or mark part of the frame, indicating something beyond the FOV. This gives the user a “peripheral indication” of the direction to turn.",
        "Breadcrumb Trails: Leave trails behind moving objects so that users can find them again if they go out of view.",
        "Though audio should rarely be used by itself to direct a user’s attention, it is powerful for reinforcing visual signals.",
        "Consider guiding people toward offscreen virtual objects. Sometimes, it can be difficult for people to locate an object that’s positioned offscreen. When this is the case, you can help people find such objects by offering visual or audible cues. For example, if an object is offscreen to the left, you could display an indicator along the left edge of the screen that guides people to point the camera in that direction.",
        "Use visual or audio cues to encourage offscreen exploration.",
        "If you know where the user is looking, you can transition their gaze to a new target location by: Moving a focus object that is easily tracked, and motivating them to track it. Having the current focus object declare the new target, and providing a trail to follow. Having the current location spawn a fastmoving object or effect in the direction of the new location, optionally with a loud or alarming sound. You can stop the motion and sound once the user has turned far enough in that direction.",
        "Mini-Map: Build a mini-map showing the room geometry and the position of virtual objects.",
        "Sound can: Guide users to next steps. (Microsoft p.129)",
        "You can spatialize audio in your application to: Call attention to visual elements. (Microsoft p.129)",
        "Proper design of interactions and content within an experience should encourage the user to move around their space, directing their attention to key content, and helping see the full potential of mixed reality. (Microsoft p.114)",
        "Leverage Human Perception: Cue users where their attention should go",
        "Peripheral Glint: Offer a peripheral glint or aura at the edge of FOV to indicate the presence of out-ofview object(s)."
    ],
    "Audio": [
        "Use audio cues to enhance the user’s experience and encourage engagement. Audio encourages users to engage with the app and explore the 360 degree environment. Ensure that your audio adds to the experience rather than distracting from it.",
        "If you’re using audio for 3D objects or the 360 environment, be mindful of a few things: - Avoid playing sounds simultaneously - Add attenuation to moderate sound effects - Set the audio to fade or stop if user is not interacting with the object - Allow users to manually turn off the audio for individual objects",
        "Spatial audio is about more than direction. Other dimensions include occlusion, obstruction, reverb, portalling, and source modeling. Collectively these dimensions are referred to as acoustics. Without acoustics, spatialized sounds lack perceived distance.(Microsoft p.129)",
        "Sound can: Effectively combine virtual objects with the real world. (Microsoft p.129)",
        "Acoustics more deeply connect holograms to the mixed-reality world. It provides cues about the environment and object state.(Microsoft p.129)",
        "Audio cues are powerful. Spatial cues augment our visual experience. Your eyes only see one place at a time. Your brain track out-of-sight areas by their sounds. Spatial cues can trigger users to turn their attention in a direction."
    ],
    "Audio Feedback": [
        "You can also communicate scale with sound effects. Alter the scale and pitch of a sound to deepen the user’s immersion in your experience.",
        "Sound can: Increase user confidence in gesture and voice interactions. (Microsoft p.129)",
        "The sound effect should be added to enhance feedback when the button is triggered (Microsoft p.16)"
    ],
    "Avatare": [
        "Use unrealistic avatars. Realistic avatars can fall into the uncanny vally. We suggest using this opportunity as a moment for creative brand expression."
    ],
    "Choice of Interaction paradigm": [
        "Where possible, open the system keyboard immediately when a cursor activates an input field, and give it auto-focus. Suspend other in-app interactions or distractions when the system keyboard is open so users can complete the text entry task unrushed.",
        "Consider whether your app needs gestures and use gestures sparingly.",
        "If the requirement of the application is that it will always be used hands-free, whether by using dwell, custom voice commands, or the single voice command, select, then make sure to make the appropriate accommodations in your UI.(Microsoft p.195)",
        "If your app needs fine manipulation with the Touchpad, consider enabling a modal interaction so users don’t have to hold the Trigger continuously. This requires less dexterity.",
        "Users interacting with digital content in their real world have less mental bandwidth for controllers and interfaces. Reduce inputs requirements, or use simple gesture-based interactions to simplify the experience.",
        "Design for anticipated detours. When using inputs with lower technological readiness, design for graceful recovery from an error. For example, degrade to headpose when the system reports a bad eye calibration",
        "Use the Whole Interface: Consider how a gesture, tap or voice can feel more powerful or magical than a button-press.",
        "Eye-gaze is not the same as Head-gaze: Consider whether fast yet ragged eye movements fit your input task.",
        "If the user needs to input text into your application, consider using a speech-to-text interaction.",
        "If a user isn't able to move around, give them an alternative way to use your app. For example, when the user is supposed to move closer to a target, give another way to access the target. Whenever it’s possible, let users tap objects and move them closer, or offer a reticle to help users reach faraway objects. Place text and instructions so they’re visible from every angle",
        "Provide a fallback input method using the Control. Map gesture interactions to the Control where it makes sense to do so.",
        "You can enable unique input methods for user interaction, but you should mirror these to standard Control actions whenever possible.",
        "Develop for the lower numbers [DOF] if your users must also navigate the real world. Using more natural inputs can help. If you’re using real world manipulation instead of 3D controllers, the experience becomes more accessible.",
        "Realistic interactions, intuitive controls using real world metaphors, and common body movements take advantage of our brains' expectations and simplify tasks. But this modeling must align with expectations to work well."
    ],
    "Coaching View for Detection": [
        "Show users how to find a surface using their phone. Use illustrations or animations to show users how to scan properly. This can help users understand to hold their phone toward the surface that they’re trying to detect, to move the phone in a clockwise or circular motion, and to search for surfaces that aren’t too dim or too bright",
        "Using Coaching to Get People Started Before people can enjoy an AR experience in your app, they need to move their device in ways that lets ARKit evaluate the surroundings and detect surfaces. In iOS 13 and later, you can use the built-in coaching view to show people what to do and provide feedback during the initialization process. You can also use the coaching view to help people reinitialize AR — a process known as relocalization — after an AR experience is interrupted by, for example, people switching briefly to a different app.",
        "If necessary, offer a custom coaching experience. Although you can configure the system-provided coaching view to help people provide specific information — such as the detection of a horizontal or vertical plane — you might need additional information or want to use a different visual style. If you want to design a custom coaching experience, use the system-provided coaching view for reference.",
        "Hide unnecessary app UI while people are using a coaching view. By default, the coaching view appears automatically when initialization or relocalization starts, so you should be prepared to hide unrelated UI to help people focus on the coaching view’s instructions."
    ],
    "Collider for finger gestures": [
        "To implement the idea of touching holograms directly with hands, ideally, five colliders could be attached to the five fingertips of each hand skeletal model. However, due to the lack of tactile feedback, ten collidable fingertips can cause unexpected and unpredictable collisions with holograms. Hence, we suggest only putting a collider on each index finger. (Microsoft p.161)"
    ],
    "Color": [
        "Color uniformity Because of the characteristics of holographic displays, a large bright area on the display can become blotchy. By using dark color schemes, you can minimize this issue. (Microsoft p.153)",
        "Hand occlusion Bright color makes it difficult for the user to see their hands when directly interacting with objects. Since the user cannot see their hands, it becomes difficult to perceive the depth/distance between the hand/finger to the target surface. The Finger Cursor helps compensate for this issue, but it can still be challenging on a bright white surface. (Microsoft p.152)"
    ],
    "Consider and show User's required Effort": [
        "Try to match the users intended level of effort. Most AR experiences in Spark take place on a mobile device. We also know that when social media is consumed on a mobile device it is mostly in a “lean-back”, passive consumption manner. Be mindful therefore of how much effort you are asking users to exert to use your AR experience.",
        "Offer alternative levels of effort. Consider offering options for different levels of effort, so if people want to switch their effort level based on their mindset or context they can do this easily. For instance, design the experience so that people can achieve a goal in a short time or indulge in a more extensive experience as they wish.",
        "Be clear about how much effort is required. Set expectations during setup by spelling out how much effort the user will need to make throughout the experience. For example, use animations to illustrate the movement required of the user.",
        "Consider the value/effort trade-off. Always be aware of how much value a person is getting from the experience compared to the amount of effort they are making. If they need to put in too much effort for too little reward then they may question why – or ultimately abandon the experience."
    ],
    "Consistency": [
        "Be reliable. Use global, consistent interactions to make your app both easy to use and safe to explore.",
        "System icons provide users with immediately recognizable visual queues and information. You can use the following icons are available for use in your apps and experiences.",
        "Use consistent, clear, and meaningful symbols.",
        "Certain Control actions must be familiar, intuitive, and adhere to platform conventions",
        "If you use a control ray for selecting things, make sure all your menus work well with the ray. Your user is likely to be confused if they have to switch to another input mechanism such as swiping on the touchpad.",
        "Don’t force users to learn a new pattern specific to your app for common interaction when the standard pattern is sufficient",
        "Use familiar UI patterns. Take advantage of your users’ knowledge. If there’s a standard UX interaction model for a certain action, such as tapping or dragging, use it! You won’t have to teach the user a whole new way to perform simple tasks, and you can dive right into the important part of your experience.",
        "Aim for visual consistency. The visuals used for instructions, surface detection, and within the experience itself should share a single consistent look. Aim for visual harmony in all parts of your experience"
    ],
    "Content Placement": [
        "Set spatial constraints. Don't let users push items too far away from themselves. If an object gets too far away, a user may have difficulty getting an object back.",
        "Too close for comfort. Avoid allowing content to get too close to your user. Magic Leap clips content at a supernear depth (~.37m) to avoid the possibility of eye strain, and this can ruin the mixed reality illusion.",
        "While a distance of 1.5 m is typically comfortable, users with poor eyesight might need to get closer or further from text to read it. Don’t fight them if they try to adjust their distance.",
        "Never fill the field of view with content",
        "Initial placement of content is crucial in creating a comfortable experience. Content placed in awkward or difficult locations causes discomfort.",
        "Place the content’s focal point at a comfortable distance from the user. Two planes are optimal for content placement: 50cm and 1.5m.",
        "Designers creating mixed reality content need not limit their experience to directly in front of the user and within their immediate view. If the physical world around the user is mapped, then all these surfaces should be considered a potential canvas for digital content and interactions. (Microsoft p.114)",
        "Although content may occasionally need to be displayed closer than 1m, we recommend against ever presenting holograms closer than 40cm. Thus, we recommend starting to fade out content at 40cm and placing a rendering clipping plane at 30cm to avoid any nearer objects. (Microsoft p.98)",
        "For maximum comfort, the optimal zone for hologram placement is betw een 1.25m and 5m. In every case, designers should attempt to structure content scenes to encourage users to interact 1m or farther away from the content (e.g. adjust content size and default placement parameters). (Microsoft p.98)",
        "You can set a maximum placement distance for objects. This helps to make sure that users will place objects at a comfortable viewing distance. It also guarantees that the object will stay at a realistic scale as the user continues to move it",
        "Design spatially. Distances are critical to people’s perception of how they can interact with objects in an AR scene. When designing scenes and interactions, think about how space and distance affects how people interact with objects and other people in the scene. It’s important to note that the level of comfort may differ across different cultures",
        "Introduce new content at a comfortable distance from user, at approximately 0.75m - 3.5m away, depending on the size and quality of the content.",
        "Keep all UI elements at least 0.75m away from the user. One way to ensure UI elements are the same (comfortable) distance from the user, and still easy to access, is by curving the UI window around the user",
        "Consider designing a minimum allowable distance between content and user. This can be helpful for preventing users from moving too close to an object or another user",
        "Set a maximum and minimum object placement distance in your experience. This will help objects to be placed in an area that is comfortable for the user to manipulate, and will reduce the likelihood of objects becoming lost in the distance."
    ],
    "Control Placement in Screen Space": [
        "If additional information or controls are necessary, consider displaying them in screen space. Content in screen space appears fixed to a consistent location either in the virtual world or, less commonly, on the device screen. It’s typically easy for people to find and view content in screen space because it remains stationary while the underlying AR environment moves with the device.",
        "Avoid placing controls in areas that have been defined as “title safe”. These are usually dedicated to logos or page titles",
        "Thumb reach across a screen of a mobile device can affect the AR experience. If people are asked to reach too far across the screen it can result in jerking the phone and this can consequently cause the camera to lose focus on an object."
    ],
    "Cursor": [
        "The cursor can be represented by different icons to convey possible actions a hologram can perform, such as scaling or rotation. Only add extra information on the cursor if it means something to the user. Otherwise, users might either not notice the state changes or get confused by the cursor.(Microsoft p.272)",
        "Although there is no one right cursor shape, we recommend that you use a directionless shape like a torus. A cursor that points in some direction (For example, a traditional arrow cursor) might confuse the user to always look that way. An exception to this is when using the cursor to communicate interaction instruction to the user. (Microsoft p.272)",
        "The cursor should be no larger than the available targets, allowing users to easily interact with and view the content. Depending on the experience you create, scaling the cursor as the user looks around is also an important consideration. For example, as the user looks further away in your experience, the cursor should not become too small such that it's lost. When scaling the cursor, consider applying a soft animation to it as it scales to give it an organic feeling. Avoid obstructing content. Holograms are what make the experience memorable and the cursor should not be taking away from them.(Microsoft p.272)",
        "If your application uses spatial mapping, then your cursor could align and hug every surface it sees. This gives feedback to the users that HoloLens and your application can see their space. This reinforces the fact that holograms are real and in our world and helps bridge the gap between real and virtual. If your experience is focused on a single hologram, your cursor should align and hug only that hologram and change shape when you look away from that hologram. This can convey to the user that the hologram is actionable and they can interact with it.",
        "Have an idea of what the cursor should do when there are no holograms or surfaces in view. Placing it at a predetermined distance in front of the user is one option.(Microsoft p.272)",
        "Eye-gaze is not the same as Head-gaze: Avoid attaching something directly to the user ’s eye-gaze (e.g., a slider or cursor)",
        "Do not show a cursor: While it is nearly impossible to interact without a cursor when using head gaze, the cursor becomes quickly distracting and annoying when using eye gaze. Instead of relying on a cursor to inform the user whether eye tracking is working and has correctly detected the currently looked at target, use subtle visual highlights (Microsoft p.224)"
    ],
    "Customization": [
        "Help users tell their own story, rather than determining it for them. For example, use simple default effects to help users get started quickly but still allow for further customization."
    ],
    "Design spaces": [
        "Upstage, farthest from the user: If the object is placed upstage, it’s difficult for users to see the difference between a very small object and an object that’s sitting far away. It’s useful to encourage exploration and movement, but can create a visually confusing experience",
        "Downstage, closest to the user: If an object is placed unexpectedly close to the user, it forces the user to move back, since it’s harder to see the full object. This is useful when you want the user to look down or examine something closely. However, be careful never to encourage the user to move backward! They could trip on real objects behind them",
        "When you create your experience, consider the importance of depth. Think about how users will see your content, and whether or not they’ll be able to access it. Think of the scene in 3 distinct regions: downstage, upstage, and center stage",
        "Center stage: This is the most comfortable viewing range for users. Objects here are optimal for users to interact with",
        "The intimate space is the space closest to the user's camera. Objects in this space may be dificult to interact with due to their close proximity, and may interfere with onscreen controls. The user is likely to collide with objects here, so it should be avoided for placement",
        "The personal space is a space in which users can clearly see the object and can also interact with it with few limitations. Objects at the center of focus of an AR experience should be placed in this zone. The space is more comfortable for the user alone, and asking another person to enter this space will cause social discomfort",
        "The social space is in an area which users are likely to feel comfortable to explore as it requires little movement. This space is also interpersonal – it works well for AR experiences with others, and is where users feel comfortable to interact with images/ real objects in the world",
        "The public space is the space furthest away from the user that’s still visible. Interacting with and viewing objects in this space is more limited and therefore the items that exist here must be simple, clear, and easy to understand",
        "People's comfort will change as they move between spaces. Let people know when they are moving between the personal and social space. Naturally, the experience will become less private when another person is introduced"
    ],
    "Device support": [
        "Offer AR features only on capable devices.",
        "If your app's primary purpose is AR, make your app available only to devices that support ARKit. If your app includes features that require specific AR capabilities, or if AR features are optional in your app, don’t show people an error if they try to use these features on a device that doesn’t support them; instead, simply avoid offering the feature on an unsupported device"
    ],
    "Encourage to explore": [
        "Hide objects behind virtual or physical objects. It’s a fun way to encourage the user to move",
        "Visual cues can nudge users to explore a more robust AR world around them. For example, have a bird fly off the screen until the user follows it. This movement can help direct the user to complete the intended goal",
        "If your experience demands exploration, remind users they can move around. In those cases, give users a goal that can only be completed by moving. Place a virtual object on the edge of the screen, encouraging them to move toward it",
        "Try not to require movement until it’s necessary. Getting users to move is a great way to engage them, but let them ease into the experience",
        "Perhaps the most important technique to encouraging movement and exploration within an app is to let users adjust to the experience. Give users a short period of ‘task-free’ time with the device. This can be as simple as placing an object in the space and letting users move around it or narrating an introduction to the experience. This time should be free of any critical tasks or specific gestures (such as air-tapping), instead the purpose to let users accommodate to viewing content through the device before requiring interactivity or progressing through the stages of the app. If this is a user’s first time with the device this is especially important as they get comfortable seeing content through the holographic frame and the nature of holograms. (Microsoft p.114)",
        "Consider encouraging full-body movement, rather than just head movement, by placing objects around the user's space.(Microsoft p.102)",
        "If your app encourages people to move, introduce motion gradually. For example, you might not want to make people dodge a virtual projectile as soon as they enter your AR game. Give people time to adapt to the AR experience in your app and then progressively encourage movement.",
        "Let Users Move: The strongest potential of spatial computing lies in fully committing to the experience - walking around, seeing content from all angles, entering a changed reality - not just watching it.",
        "Tempt users to explore where relevant. Interacting in a 3D space via a mobile device is a relatively new behavior for many people. While some experiences only require minor hand movements, others might require more physical movement and gestures than people are used to. Ease people into these new behaviors with prompts to explore their environment and hints to aid this exploration.",
        "Take advantage of the depth of the experience. Encourage people to carefully move forward into the scene in order to interact with objects that are further away. For instance, during way-finding experiences, present arrows in the distance to nudge people deeper into the scene"
    ],
    "Ergonomics (avoid head & neck fatigue)": [
        "Place content at a comfortable height in front of the user. Your user’s head and eyes naturally tilt downward 10 to 15 degrees and are rarely at the horizon.",
        "Don’t require your user to switch focus from object to object frequently.",
        "Headpose should be used in combination with other targeting methods. Heavy use of headpose targeting can cause users fatigue or social awkwardness.",
        "Be mindful of a person's ability to carry unsupported weight on their head or in their hands for extended periods. Allow pausing the experience at arbitrary times so people can take breaks.",
        "The optimal (resting) gaze angle is considered between 10-20 degrees below horizontal, as the head tends to tilt downward slightly, especially during activities. (Microsoft p.102)",
        "To avoid eye and neck strain content should be designed so that excessive eye and neck movements are avoided. Avoid gaze angles more than 10 degrees above the horizon (vertical movement) Avoid gaze angles more than 60 degrees below the horizon (vertical movement) Avoid neck rotations more than 45 degrees off-center (horizontal movement) (Microsoft p.102)",
        "Experiences with moving objects or large objects should pay special attention to head movement, especially where they require frequent movement along both the horizontal and vertical axes.(Microsoft p.102)",
        "Create ergonomic experiences. The “working zone” is the area where users spend most of their time consuming information and interacting with content. Be sure to minimize head movement and use easy gestures to avoid fatigue."
    ],
    "Ergonomics (avoid muscle fatigue)": [
        "Make easy transitions from one pose or movement to another",
        "Design for comfort. Try to avoid making the user do anything that’s physically demanding, uncomfortable, or too sudden",
        "Offer different levels of movement for users to achieve their goal. Sitting or standing with little movement is more comfortable and familiar for people. However, if you ask people to sit and move their arms or stand and move their body, be aware that such repetitive movement can be tiring",
        "Design controls and inputs with comfort in mind. When designing an AR experience, it is important to consider ergonomics.",
        "In room-scaled VR experiences, while a full range of movement could be explored, keep in mind that people can get tired after standing for too long. If the experience is intended for long periods of productivity, consider giving users the option of sitting down.",
        "Muscle fatigue can accumulate when users are expected to keep a hand raised throughout the duration of an experience. It can also be fatiguing to require the user to repeatedly make air tap gestures over long durations. We therefore recommend that experiences avoid requiring constant, repeated gesture input. This goal can be achieved by incorporating short breaks or offering a mixture of gesture and speech input to interact with the app. (Microsoft p.102)",
        "Get comfortable Holding your phone for a prolonged period of time can be tiring. Encourage users to move around their phone or change the position they’re holding it in. You can also build resting points into the play cycle",
        "Avoid physically challenging input",
        "Be mindful of people's comfort. Holding a device at a certain distance or angle for a prolonged period can be fatiguing. To help avoid causing fatigue, consider placing objects at a distance that reduces the need to move the device closer to the object; in a game, consider keeping levels short and intermixed with brief periods of downtime",
        "Do not force users to lift their hands for prolonged periods of time."
    ],
    "Ergonomics (avoid visual discomfort fatigue)": [
        "Ensure users can view content comfortably with enough contrast and without strobing or harsh animations",
        "Typical users find a line length of 50-60 characters per line comfortable. Long lines of text can cause eye strain and might exceed your user’s field of view, occluding text and forcing them to turn their heads to read, increasing fatigue.",
        "Use dark color for the UI background: By using the dark color scheme, you can minimize eye fatigue and improve the confidence on direct hand interactions. (Microsoft p.153)",
        "Eye fatigue: Since holographic display is additive, bright color uses more light to display holograms. Bright, solid color in a large area of the display can easily cause eye fatigue for the user. (Microsoft p.152)",
        "In other words, it is much more comfortable to look at a stationary hologram that stays 50cm away than to look at a hologram 50cm away that moves toward and away from you over time. (Microsoft p.97)",
        "Objects that move in depth are more likely than stationary objects to produce discomfort due to the vergenceaccommodation conflict. Similarly, requiring users to rapidly switch between near-focus and farfocus (e.g., because of a pop-up hologram requiring direct interaction) can cause visual discomfort and fatigue. Therefore, extra care should be taken to minimize how often users are: viewing content that is moving in depth; or rapidly switching focus between near and far holograms. (Microsoft p.98)"
    ],
    "Error prevention & recovery": [
        "Allow people to experiment without fear of breaking the experience. Design the experience in a way that allows people to try things out freely, but enables them to go back to where they were easily if they make a mistake.",
        "Even though users can stick their phones inside objects in your virtual world, it breaks the reality you’re creating and the immersive quality of the scene. When that happens, let users know they aren’t supposed to be there. Visual cues can help users avoid these unintended collisions. Make an object blurry inside when you want to say, You're not supposed to be here!",
        "When users encounter environmental limitations, indicate what went wrong and point them in the right direction",
        "Help users easily recover from missteps and errors.",
        "Whether the error came from the system or the user, make it easy to get back into the experience. Use a combination of visual cues, animation, and text to show a clear path to resolution",
        "Provide feedback when something has gone wrong, and give the user an actionable path to resolution. Give simple, short tasks. Walk the user through correcting the error state, one step at a time",
        "You can communicate what went wrong, especially if it helps avoid that error in the future. Avoid blaming the user. Focus on getting the user to take the right action. Sample error states can include: A dark environment: Too dark to scan. Try turning on the lights or moving to a well-lit area. User moving device too fast: Device moving too fast. Try moving more slowly. User blocking the sensor or camera: Looks like the sensor is blocked. Try moving your finger or adjusting the device’s position.",
        "Suggest possible fixes if problems occur. Analysis of the real-world environment and surface detection can fail or take too long for a variety of reasons — insufficient light, an overly reflective surface, a surface without enough detail, or too much camera motion. If your app is notified of these problems, use straightforward, friendly language to offer suggestions for resolving them.",
        "Indicate when the front-facing camera is unable to track a face for more than about half a second. Use a visual indicator to indicate that the camera can no longer track the person’s face. If you need to provide text instructions in this situation, keep them to a minimum"
    ],
    "FOV": [
        "Avoid Large Objects: Objects that obscure the entire FOV will noticeably cut off and break user immersion.",
        "Don’t Move Content Too Fast: Fast-moving objects run out of view very quickly and are difficult to track.",
        "For now the field of view (FOV) is limited. Careful design and sensitivity to your user’s focus can minimize their awareness of those limits. Control breadth of focus, and move at a speed users can comfortably track with head and eye movement. This helps users bring the FOV along with them.",
        "Text should appear in your user’s field of view at natural gaze height based on headpose",
        "When possible, limit the majority of interactions to either horizontal or vertical categories, ideally with most experiences taking place in the center of the holographic frame while the user's head is in a neutral position.(Microsoft p.101)",
        "Experiences with many objects or components should consider using the full space around the user to avoid cluttering the holographic frame directly in front of the user. In general, it is good practice to introduce content to an experience slowly and this is especially true with experiences that plan to serve many objects to the user. Much like with large objects, the key is to let users understand the layout of content in the experience, helping them gain a spatial understanding of what’s around them as content is added to the experience.(Microsoft p.114)",
        "Horizontal head movement is more for frequent interactions, while vertical movements should be reserved for uncommon events. For example, an experience involving a long horizontal timeline should limit vertical head movement for interactions (like looking down at a menu).(Microsoft p.102)",
        "Objects that cannot normally fit within the holographic frame should be shrunk to fit when they are first introduced (either at a smaller scale or at a distance). The key is to let users see the full size of the object before the scale overwhelms the frame. (Microsoft p.114)",
        "Don't expect users to look behind them. People find it hard to think about more than one space at a time and will focus on what is in their current field of view.",
        "People can easily forget where they have placed objects when they are no longer in their FOV. Remind people where objects are placed periodically, as part of the overall experience.",
        "Avoid spreading objects outside of the FOV upon launch of the experience. If you do, be aware that it will take considerable effort for people to discover them, plus this level of movement might be awkward when in a public space.",
        "Design through the camera’s FOV. The Field of View (FOV) is the observable area that the camera can see and thus the view that a user can see through their device. It is narrower than the FOV a person can see from their own eyes."
    ],
    "Feedback": [
        "Use visual cues for feedback as your baseline AR is mostly a visualled experience, therefore most baseline feedback should exist in a visual format by default. However, visual feedback can also be supplemented with other types of feedback in order to provide a more engaging and immersive experience while simultaneously increasing the accessibility for everyone.",
        "It is vital to show feedback to your user as they interact with your app. It increases the realism and immediacy of the experience and will ensure that the user doesn't think there are errors or problems with the app or controls.",
        "As there is no physical feedback, take care to provide ample visual and audio feedback during and following a gesture. Providing clear, continuous, and responsive feedback will also help your user understand that the system is recognizing a meaningful gesture.",
        "We naturally coordinate our headpose and eye gaze to direct our focus. What a user is looking at may not be what their headpose ray is intersecting. Give users clear feedback for what their headpose targeting so they can compensate.",
        "In the real world when actions take place, they have physical responses. Use these as inspiration to inform your user of actions they have taken.",
        "Feedback can come in many forms, such as onscreen visual feedback, haptics, audio, or LED indicators. Combining multiple forms of feedback can significantly enhance your user's experience.",
        "Feedback should always be immediate, minimize lag, be appropriate to the action, and not overwhelming."
    ],
    "Fitt's Law for Touch Interaction": [
        "Consider the sizing of touch targets to allow for easy interactions.",
        "When your app detects a gesture near an object, assume that a user is attempting to interact with it. Provide reasonably sized touch targets despite the object’s small size.",
        "Respond to gestures within reasonable proximity of interactive virtual objects. It can be difficult for people to be precise when aiming to touch specific points on objects that are small, thin, or placed at a distance. When your app detects a gesture near an interactive object, it's usually best to assume that people want to affect that object."
    ],
    "Hand Menus": [
        "Keep the number of buttons small (Microsoft p.321)",
        "Avoid adding buttons near the wrist (Microsoft p.321)",
        "Utilize hand menu for quick action (Microsoft p.321)"
    ],
    "Handling Interruptions / Relocalization": [
        "Minimize interruptions if your app supports both AR and non-AR experiences. One way to avoid interruptions is by embedding a non- AR experience within an AR experience so that people can handle the task without exiting and reentering AR. For example, if your app helps people decide on a piece of furniture to purchase by placing the item in a room, you might let them change the upholstery without leaving the AR experience.",
        "ARKit can't track device position and orientation during an interruption, such as when people briefly switch to another app or accept a phone call. After an interruption ends, previously placed virtual objects are likely to appear in the wrong real-world positions. When you enable relocalization, ARKit attempts to restore those virtual objects to their original real-world positions using new information.",
        "Allow people to cancel relocalization. If people don’t position and orient their device near where it was before an interruption, relocalization continues indefinitely without success. If coaching people to resume their session isn’t successful, consider providing a reset button or other way to restart the AR experience.",
        "Consider using the system provided coaching view to help people relocalize. During relocalization, ARKit attempts to reconcile its previous state with new observations of the current environment. To enable these observations, you can use the coaching view to help people return the device to its previous position and orientation.",
        "Consider hiding previously placed virtual objects during relocalization. To avoid flickering or other unpleasant visual effects during relocalization, it can be best to hide virtual objects and redisplay them in their new positions.",
        "Design seamless transitions. When the user moves quickly, ARCore might lose its tracking. Design smooth, subtle transitions between surface finding and detection",
        "Your app should tell the user how to get positional tracking back, rendering some fallback body-locked content that describes some tips, such as uncovering the sensors and turning on more lights. (Microsoft p.108)"
    ],
    "Haptic Feedback (phones)": [
        "Due to the variation in technology between different Android phones, haptic feedback can be unpredictable. In some cases, it can cause AR tracking to be lost. To create the effect of haptic feedback, you can use low-frequency sounds.",
        "Phone vibrations are frequently used to send feedback to users. However, haptic feedback isn’t recommended for AR experiences."
    ],
    "Hardware Properties": [
        "It’s a good practice to research and follow device-specific guidance whenever possible. Don’t fight the limitations of your hardware.",
        "HoloLens displays are fixed at an optical distance approximately 2.0m away from the user. Thus, users must always accommodate near 2.0m to maintain a clear image in the device. .. (Microsoft p.97)"
    ],
    "Image detection": [
        "Limit the number of reference images in use at one time. Image detection performance works best when ARKit looks for 100 or fewer distinct images in the real-world environment. If you need more than 100 reference images, you can change the set of active reference images based on context. For example, a museum guide app could ask permission to use location services to determine the part of the museum a person is in, and then look only for images displayed in that area.",
        "Limit the number of reference images requiring an accurate position. Updating the position of a reference image requires more resources. Use a tracked image when the image may move in the environment or when an attached animation or virtual object is small compared to the size of the image",
        "You can enhance an AR experience by using known images and objects in the real-world environment to trigger the appearance of virtual content. For example, an app that recognizes theater posters for a sci-fi film could cause virtual space ships to emerge from the posters and fly around the environment. Another example is an app for an art museum that presents a virtual tour guide when it recognizes a sculpture. To enable experiences like these, your app provides a set of 2D reference images or 3D reference objects, and ARKit indicates when and where it detects any of these items in the current environment.",
        "When a detected image first disappears, consider delaying the removal of virtual objects that are attached to it. ARKit doesn’t track changes to the position or orientation of each detected image. To help prevent virtual objects from flickering, consider waiting up to one second before fading them out or removing them."
    ],
    "Immersion": [
        "Use audio and haptics to enhance the immersive experience. A sound effect or bump sensation is a great way to confirm that a virtual object has made contact with a physical surface or other virtual object. Background music can also help envelop people in the virtual world. For guidance, see Audio and Haptics."
    ],
    "Inform about Waiting Time": [
        "Be transparent and manage expectations [waiting time]. Clearly communicate how long the action might take.",
        "Be playful with wait times. Minimize frustration by designing in an engaging distraction (i.e. games or gifs) where appropriate.",
        "Understand and minimize wait times. People are more likely to give up the longer they have to wait. People's device and level of connectivity will impact the performance of the experience. When people do have to wait, help them understand why and use techniques to engage and/or distract them."
    ],
    "Information revealing": [
        "If necessary, provide a way to get more information. Design a visual indicator that fits with your app experience to show people that they can tap for more information.",
        "Minimize text in the environment. Display only the information that people need for your app experience.",
        "Present the right information at the right time. An exciting feature of AR is that content can change and be interactive in real time. We can take advantage of these features in the way we present information to users, whether that is instructions or narrative.",
        "Explore using contextual cues tagged to a location or time. This will help users who may be struggling and in need of further assistance."
    ],
    "Instructions": [
        "A device's position impacts the detectable depth. The perceived depth of a scene will change depending on the angle at which the user is holding their phone. Encourage users to change the angle of their phone if your AR experience requires more space or depth in the scene.",
        "Your instructions are a guide. Demonstrate and show, rather than tell.",
        "Ease users into instructions. Break instructions into bitesize chunks to avoid cognitive overload and only present information when a user needs to act upon it (i.e. avoid putting all of the instructions at the beginning of the experience).",
        "Alerts should be actionable. Provide a user instruction and ability to resolve alerts.",
        "Let users know what movements will trigger the app",
        "In a three-dimensional context, prefer 3D hints. For example, placing a 3D rotation indicator around an object is more intuitive than displaying text-based instructions in a 2D overlay. Avoid displaying textual overlay hints in a 3D context unless people aren’t responding to contextual hints.",
        "Drag to place an object. After selecting a virtual object by touching it, users can drag the object into their scene. Many users might not know how to drag. Give clear instructions, and walk them through the process of dragging step by step",
        "Guide them through the types and range of movement possible",
        "Illustrate instructions, rather than only spelling out in text. This is especially helpful if a person needs to perform a gesture or overcome an error",
        "Guide the user visually. Use a combination of visual cues, motion, and animation to teach users. Illustrate and use in-app experiences as much as possible. Text instructions can take users out of the experience and make it harder to remember what they’re supposed to do. For example, if you want users to swipe, give them an arrow or a hand icon rather than showing the word “swipe.”"
    ],
    "Keep the focus on AR experience, but use 2D-UI On-Screen elements when needed": [
        "Consider using indirect controls when you need to provide persistent controls. Indirect controls are not part of the virtual environment — instead, they are 2D controls displayed in screen space. If people need access to persistent controls in your app, consider placing the controls so that people don’t have to adjust how they're holding the device to reach them. Also, consider using translucency in an indirect control to help avoid blocking the underlying scene. For example, the Measure app uses screen space to display a mix of translucent and opaque controls that people use to measure objects in the real world.",
        "Let people use the entire display. Devote as much of the screen as possible to displaying the physical world and your app's virtual objects. Avoid cluttering the screen with controls and information that diminish the immersive experience.",
        "It’s best to keep the user focused on the AR experience itself. Sometimes, however, an app needs to have onscreen controls.",
        "In those cases, make the controls as simple as possible. Ideally, a user should be able to trigger an action without looking at it. Think of the camera button on your phone. It’s big, it’s not labeled, and you can tap it almost without thinking about it.",
        "Create a world that’s immersive and easy to use. Immerse users, don’t distract them. Try to interrupt your AR world as little as you can. Get users into the experience, and then get out of the way. Avoid pop-ups and fullscreen takeovers unless the user explicitly selects it. Buttons, 2D alerts, and notifications can distract the user from the 3D world that you’re creating around them. Instead, let users focus on the scene itself. Persistent 2D overlays can also disrupt the user’s immersion. It’s a constant reminder that the world they’re looking at isn’t completely real.",
        "Avoid pop-ups and full-screen takeovers unless the user explicitly selects it. Buttons, 2D alerts, and notifications can distract the user from the 3D world that you’re creating around them. Instead, let users focus on the scene itself.",
        "Maintain the continuity of the experience. Try to avoid taking the user out of a scene too often. For instance, if users need to select, customize, or share an AR object, try to figure out a way they can do it without leaving AR.",
        "Smart notifications: Taking into account what a user is paying attention to, you can make this experience better by offsetting notifications from where the user is currently gazing. This limits distractions and automatically dismisses them once the user is finished reading. (Microsoft p.111)",
        "Only show control options when required. For example, only show a trash icon when a user has selected an object and wants to delete it from the scene.",
        "People may struggle to interact with on-screen controls as they are moving. Be aware that this can make it dificult for them to accurately use the controls",
        "Try not to overlay modals that obstruct the real-word view for the whole experience. It is important to understand the right balance between carrying out focused tasks and being immersed in the augmented world for your AR experience",
        "Try to limit the use of on-screen controls and UI 2D on-screen controls and UI take up screen space on a mobile device, interrupting the user's immersion. It is important that these controls are minimized where possible as too many controls can mask how much AR content can be viewed on the screen.",
        "Limit what is presented on screen. Only show frequently used controls continuously, and hide other controls or use collapsible menus to reveal more controls only when they are needed."
    ],
    "Landscape / Portrait Mode": [
        "Provide support for both portrait and landscape modes. If this isn’t possible, select the one that’s best for your experience. Supporting both modes creates a more immersive experience and increases user comfort.",
        "Think about camera and button placement for each mode. Pay attention to how camera positioning affects depth sensing, spatial awareness, and accurate surface measurements in each mode."
    ],
    "Language": [
        "If you must display instructional text, use approachable terminology. AR is an advanced concept that may be intimidating to some people. To help make it approachable, avoid using technical terms like ARKit, world detection, and tracking. Instead, use friendly, conversational terms that most people will understand.",
        "Use clear and simple language As AR is still being adopted by the mainstream, many of the terms are still being defined and most users will not be familiar with or nterested in the technical aspects of AR.",
        "Use clear and understandable language. Your user needs to understand what they are agreeing to. Being vague or overly technical in your explanation doesn't help a user know why you need access to a privilege."
    ],
    "Law of practice": [
        "Cater to both the expert and the novice. Let users fast track or skip past instructions depending on their level of comfort and expertise.",
        "Be mindful of peoples different levels of expertise and design the experience to ensure a novice feels minimal friction during their learning curve and that an expert experiences minimal frustration.",
        "Design for flow. Integrate targeting, activation, and manipulation into seamless interactions. Create a smooth path from novice to expert usage.",
        "Remember your users' familiarity with mixed reality when making your experience."
    ],
    "Magic Leapspecific Styleguide": [
        "Avoid rectangular frames and simplify interfaces.",
        "Ensure your immersive app presents a clear exit or quit option when users tap the Home button."
    ],
    "Manipulation gestures": [
        "Let people directly interact with virtual objects using standard, familiar gestures. For example, consider supporting a single-finger drag gesture for moving objects, and a two-finger rotation gesture for spinning objects.",
        "Support both 1- finger and 2- finger gestures for manual rotation.",
        "To avoid conflicts with translation, 1- finger rotation should only happen when the swipe gesture starts off the selected object.",
        "To start moving an object, the user selects it. They can either drag the object along the screen with one finger, or they can move the actual phone, and the object will follow.",
        "Two-finger gestures are commonly used to rotate or scale an object. These can include: Rotating an object with index finger and thumb (on the same hand) Touching the screen with thumb and index finger, holding your thumb still on the object and rotating it with your index finger Rotating an object with two thumbs (using two hands)",
        "To avoid conflicts with scaling, 2-finger rotation should only happen when both fingers rotate in opposite directions on a horizontal axis.",
        "Scaling lets the user increase or decrease the size of an object. It’s most often performed with a pinch gesture."
    ],
    "Multiuser": [
        "Restrict participation to identifiable people (such as company employees only).",
        "Keep the user group small. Consider capping the maximum number of people interacting at a given time.",
        "Remember that the frontfacing camera isn't just for individual use. It can also be used in more social situations to help multiple people be part of the AR experience.",
        "Remote eye-gaze visualizations: Visualize what remote collaborators are looking at, to be able to provide immediate feedback and facilitate more accurate information processing. (Microsoft p.111)",
        "Connect Locally: Think about all the ways your experience can connect people. Consider realtime competition or collaboration. Include others both with devices and without.",
        "A multiplayer experience lets different users share the same AR environment. An object that appears on one user’s device will appear to all users. Player 1 detects a surface Players 2, 3, and 4 detect the same surface by moving closer to Player 1. The app recognizes all the players and connects them! Everyone now shares the same AR environment. Multiplayer experiences can require more hand-holding than single-user journeys. Guide your users through each step. Try to make the moment of connection as seamless as possible.",
        "When more users join, encourage them to move closer to the first user. Their devices will connect faster",
        "When possible, let new participants enter a multiuser AR experience. Unless your app requires all participants to join before the experience begins, consider using implicit map merging to let new people quickly join an ongoing AR experience.",
        "Designing a Great Multiuser Experience. When multiple people share your app's AR experience, each participant maps the environment independently and ARKit automatically merges the maps"
    ],
    "Object Manipulation": [
        "Allow users to manipulate content with actions such as rotation, scale, and distance.",
        "Let the user select a virtual object in order to identify, manipulate, and interact with it.",
        "Regardless of the purpose of your app, don’t use scaling as a way to adjust the distance of an object. If you enlarge a distant object in an effort to make it appear closer, the result is a larger object that still looks far away.",
        "Support user-initiated object scaling when it makes sense in your app. For example, if your app lets people explore an imaginary environment, it probably makes sense to support object scaling because your app doesn’t need to represent the real world. On the other hand, if your app helps shoppers decide on furniture to buy, letting people scale a chair object doesn’t help them visualize how the chair will look in a room.",
        "Let people use direct manipulation to interact with objects when possible. It’s more immersive and intuitive when people can interact with onscreen 3D objects by touching them directly, than by using indirect controls in screen space. However, in situations where people are moving around as they use your app, indirect controls can work better.",
        "In general, keep interactions simple. Touch gestures are inherently twodimensional, but an AR experience involves the three dimensions of the real world. Consider the following approaches to simplifying user interactions with virtual objects: Limit movement to the two-dimensional surface on which the object rests. Limit object rotation to a single axis."
    ],
    "Object Placement": [
        "Users can manually add virtual objects to a scene and move them around. Manual placement works best when: The experience is fully interactive, such as a jigsaw puzzle. Object placement needs to be precise, such as a furniture selection app",
        "A user can drop a virtual object by tapping a spot anywhere in the scene. Tapping generally feels natural to the user, and works best when: The virtual object doesn’t need to be adjusted or resized before it’s placed. It should be placed quickly",
        "Automatic placement works best when: A virtual environment is added, without any user input, into the real-world space There’s no interaction, or minimal interaction It doesn’t matter if objects appear in exactly the right spot AR mode is crucial to your experience AR mode starts when the experience is launched",
        "When people place an object, immediately integrate that object into the AR environment. Although surface detection quickly and progressively refines accuracy, it's best to avoid waiting for more accurate data before placing an object. Use the information available to respond instantly when people place an object; then, when surface detection completes, subtly refine the object's position if necessary. For example, if people place an object beyond the bounds of the detected surface, gently nudge the object back onto the surface.",
        "Avoid trying to precisely align objects with the edges of detected surfaces. In AR, surface boundaries are approximations that may change as people's surroundings are further analyzed.",
        "Incorporate plane classification information to inform object placement. For example, only let people place a virtual piece of furniture on a plane that's classified as “floor,” or require a plane to be classified as “table” in order to place a virtual game board.",
        "Using the surface magnetism technique, you can place a holographic object on a realworld physical surface. By aligning the object's orientation with the real-world surface, you can create a more immersive and natural experience for the user. (Microsoft p.381)",
        "Help people to find lost or forgotten objects. One way to do this is to ask them to place an object into a scene. This action helps them to memorize that position, making it easier to locate the object when they can't see it."
    ],
    "Object Scaling ": [
        "Setting limits on your objects and making sure they can't be sized too large or small is important.",
        "Consider adding a bouncing effect to indicate the object has reached maximum and minimum size. You can also lock the object once it’s achieved the desired scale.",
        "For an optimal experience, add a minimum and maximum scale limit. The more precise you set the scaling values, the more precise you can control the composition of the scene.",
        "Playing with scale can add an unexpected aspect to an experience. Being surprised by a huge virtual character, depending on your intention, can be either hilariously surprising or absolutely terrifying. Depending on your goals and the effect you’re trying to create, you can use surprise as a tool to evoke different emotions, whether a horror game or a silly playground.",
        "Try to match the scale of virtual objects to the experience. People often struggle to distinguish between scaling an object in 3D and it moving further away",
        "Generally, always keep target areas at a minimum finger width. It is not always clear to users that objects will scale relative to the environment, and therefore when objects move back into the distance they perceive it as scaling and not moving",
        "Consider setting minimum and maximum sizes for all virtual objects. These values should be relative to the intended experience “size” (i.e. tabletop, room size etc) and should ensure the object controls are still usable, regardless of location",
        "Keep objects usable as they move further away. As a virtual object moves away from the viewer and into the distance of the scene, the object should reduce in size on the screen. However, it is important that the area of interaction (the target area) does not reduce with the object as people still need to be able to interact with that object using their fingers."
    ],
    "Object rotation": [
        "Avoid auto-rotating an object unless it’s an intentional part of the experience. Persistent autorotation can be disconcerting for users.",
        "Rotating a virtual object lets the user orient the object’s position in any direction. Objects can be rotated either manually or automatically."
    ],
    "Object translation (Verschiebung)": [
        "Try to avoid sudden transitions that might create the illusion of changes in scale. When an object moves too fast, and in multiple directions at once, it may appear to be shrinking or growing.",
        "Let the user move and rotate an object in case they can’t physically move around it",
        "It’s a fun way for virtual objects to interact with the real world. During the translation from one surface to another, avoid sudden transitions or changes in scale.",
        "Encourage users to move a virtual object from one surface to another."
    ],
    "Occlusion": [
        "Objects that appear too far can be hidden by real or virtual content",
        "Use depth to establish hierarchy and group content.",
        "Sometimes, occlusion of holograms is undesirable. If a user needs to be able to interact with a hologram, then they need to be able to see it - even if it is behind a real-world surface. In such cases, it usually makes sense to render such a hologram differently when it is occluded (for example, by reducing its brightness). This way, the user will be able to visually locate the hologram, but they will still be aware that it is behind something. (Microsoft p.134)",
        "Occlusion also provides information to the user; when a hologram appears to be occluded by a real-world surface, this provides additional visual feedback as to the spatial location of that hologram in the world. (Microsoft p.133)",
        "Conversely, occlusion can also usefully hide information from the user; occluding holograms behind wall scan reduce visual clutter in an intuitive way. To hide or reveal a hologram, the user merely has to move their head. (Microsoft p.133)",
        "Consider enabling people occlusion. If your app supports placing virtual objects behind people who appear in the device’s camera feed, enhance the illusion of reality by letting the people occlude the objects."
    ],
    "Onboarding": [
        "Consider introducing the user to a tutorial, or progressively revealing functionality over time.",
        "Onboarding can help people get started quickly. A good place to set people's expectations is in the onboarding process. There are likely to be key interactions that are new to people which, if introduced up front, could speed up people's progress.",
        "Help users understand AR quickly. When people interact with technology they build a mental model in their head of how it works. As AR is new to so many people, they will not have a mental model to reference. Help people to learn how AR works by keeping the experience simple and by providing hints on what the AR technology is doing – for example, showing an active state when objects can be manipulated. This could help improve the user's understanding without burdening them with technical details.",
        "Let users launch AR quickly. Make your tutorial a part of the main experience flow. Avoid teaching users all the key tasks or mechanics at once. Rather, show them how to perform these tasks as they show up in the game. Users won’t be overloaded with information, and they’ll be able to link helpful instructions and tips to the task at hand.",
        "Provide an onboarding flow within the experience"
    ],
    "Pause / Breaks": [
        "If needed, allow users to have a rest during the experience. If an experience is physically tiring or uncomfortable, then people may disengage or even abandon the experience, unless it is explicitly designed to be tiring (e.g. exercise and fitness applications, or social gaming etc)",
        "Avoid long play sessions Users can get fatigued using AR for extended periods of time. Try to find stopping points in the action or moments when users might need to take a break",
        "Take a break. Let users pause or save their progress. Make it easy to continue an experience where they left off, even if they switch their physical location",
        "As users become tired or stressed, their ability to process information and participate drops. Let users recharge from time to time.",
        "Where an immersive experience allows, pause the app when the menu is invoked to let your user address real-world interruptions."
    ],
    "Performance": [
        "Mixed reality apps are unique because users can move freely in the world and interact with virtual content like as though they were real objects. To maintain this impression, it is critical to render holograms so they appear stable in the world and animate smoothly. Rendering at a minimum of 60 frames per second (FPS) helps achieve this goal. There are some Mixed Reality devices that support rendering at framerates higher than 60 FPS and for these devices it is strongly recommended to render at the higher framerates to provide an optimal user experience.(Microsoft p.99)",
        "If your application requires the lowestlatency possible and mesh triangles that only you will want to access, use Spatial Mapping directly. If you are performing higher level processing, you may consider switching to the Scene understanding model as it should provide you with a superset of functionality. Also note that because Scene understanding provides a snapshot of the spatial mapping mesh as part of its representation, you will always have access to the most complete and accurate spatial mapping data possible. (Microsoft p.123)",
        "To let your scene load faster, don’t make textures too large. Their resolution should be 2k at most",
        "To help avoid breaking the illusion you create, make sure your app updates scenes 60 times per second so objects don’t appear to jump or flicker"
    ],
    "Permissions": [
        "Be clear about the relevance and benefits of each permission. For instance, if the app needs access to the camera for AR to work, or the user’s position for multiplayer experiences, let them know.",
        "Clearly tell users why the app needs certain permissions.",
        "Ask for permissions only when it’s necessary for users to move forward with the experience",
        "Consent is not implied. Never assume that a user will provide you permission to a privilege. Your app should fallback gracefully and retain as much functionality as possible when privileges are denied.",
        "Explain the benefits of granting privileges. Your user should be aware of why you are collecting their data and how it is being used. This can be achieved by letting a user know why a privilege is needed before requesting it.",
        "Only use the privileges you need. Only use privileges when these are necessary for your app to function."
    ],
    "Physics simulation": [
        "Strive for virtual object movement that’s consistent with the physics of your app’s AR environment. People don’t necessarily expect an object to move smoothly over a rough or uneven surface, but they do expect objects to remain visible during movement. Aim to keep moving objects attached to real-world surfaces and avoid causing objects to jump or vanish and reappear as people resize, rotate, or move them.",
        "The use of physics simulation is another way in which spatial mapping can be used to reinforce the presence of holograms in the user's physical space. When my holographic rubber ball rolls realistically off my desk, bounces across the floor and disappears under the couch, it might be hard for me to believe that it's not really there. (Microsoft p.134)",
        "Physics simulation also provides the opportunity for an application to use natural and familiar physicsbased interactions. Moving a piece of holographic furniture around on the floor will likely be easier for the user if the furniture responds as if it were sliding across the floor with the appropriate inertia and friction. (Microsoft p.134)",
        "Embrace Physicality. Respect the real world. By having your digital objects respond to basic physics in the world, you firmly ground them in reality.",
        "Embrace Physicality. Respect the real world. By having your digital objects respond to basic physics in the world, you firmly ground them in reality."
    ],
    "Shared spaces": [
        "Shared social experiences can reduce the space available to a user. Users are likely to be in close proximity of others who are part of the AR experience. Be aware that the space the user has to carry out movement may be reduced because of this.",
        "Introduce a “Personal Bubble” approximately 0.75 meter in radius in multiplayer experiences to prevent users from getting too close to each other. Alternatively, consider equipping users with a “power gesture”; an action that projects a barrier to deflect unwanted proximity."
    ],
    "Social Acceptance": [
        "Avoid gestures that could be derogatory or disrespectful out of context.",
        "Bystanders may cause discomfort. Be mindful of the actions you ask users to perform in public spaces and the social awkwardness that this may cause"
    ],
    "Space requirements": [
        "Give the user room to move and interact with their surroundings.",
        "[Users] could be at home in a quiet and calm space or in a public space surrounded by moving people and things. Consider different environments and how they might impact your design.",
        "It's still helpful to consider all the places that your app might be used, from a small apartment to a vast field. Prepare for large and small spaces, real-world obstacles like furniture or traffic, and physical challenges",
        "Give users a clear understanding of the amount of space they'll need for your app. You can use it on your lap, a kitchen table, or a football stadium? Show them the ideal conditions for using it. You can include preview graphics in the Play Store, or instructions in the app itself",
        "Your users’ spaces will vary significantly in size and complexity. While some apps work regardless of environment, always consider users’ spatial limitations.",
        "Show users the ideal size and the perfect conditions for your experience. Set their expectations right away. You can still surprise them, but let them know where those surprises might happen",
        "When you start to design your app, decide the size of the playspace. Is it the size of a tabletop, a bedroom, or as big as the world? The experience should fit the environment",
        "Always consider your users’ spaces. To avoid depth collisions—when a virtual object appears to intersect a real-world object—be mindful of reasonable room sizes and the various environments in which users could use your app.",
        "Responsive Playspace: If you’re creating a tabletop experience, make sure it fits on every tabletop. Design the experience size to be responsive. Some users might play on a large banquet table. Others might set up the experience on a small desk",
        "Content placement should be informed by the device and design intention. Before starting the project, ask yourself if the primary delivery platform of the experience is going to be room-scaled VR, Mobile VR, or seated VR. Is it going to be for people in an airplane seat who cannot freely turn around or can you assume users will be in a swivel chair?",
        "Set expectations up front by clearly communicating how much space is needed for the experience, from a tabletop, to an entire room, to world-scale space."
    ],
    "Space-Robust Applications in dynamic environments": [
        "Anticipate that people will use your app in a wide variety of real-world environments. People may open your app in a place where there isn't much room to move around or there aren't any large, flat surfaces. Clearly communicate your app's requirements and expectations to people up front to help them understand how their physical environment can affect their AR experience. You might also consider offering different sets of features for use in different environments.",
        "Sometimes, the device cannot track properly if there are lots of dynamic changes in the environment, such as many people walking around in the room. In this case, the holograms may seem to jump or drift as the device tries to track itself in this dynamic environment. We recommend using the devices in a less dynamic environment if you hit this scenario. (Microsoft p.180)",
        "Assume a dynamic environment An AR experience can be initiated in any realworld environment at anytime. This means that as a designer, we have little control of the context for the experience. We believe the best-designed AR experiences will consider, accommodate and even embrace the breadth of possible contexts.",
        "Public spaces provide their own set of challenges for AR. Tracking and occlusion become difficult, depending on the number of objects and people around. Also, phone movement and AR immersion can be distracting or dangerous",
        "Don't get trapped in the habit of always testing in the same location, for example at your desk. Make sure to test on various surfaces of different positions, shapes, sizes and materials. (Microsoft p.139)",
        "Physical environments: Design one experience for many different spaces",
        "A user's location may change. They could be in a relatively static location or on the move, and range from a user's living room to a bus or busy street – each of which has its own safety considerations. For example, using a phone's accelerometer, you can tell whether a person is walking, allowing you to add a prompt telling them to look up from their phone.",
        "Crowded environments need special consideration and planning. This increased cognitive load can be from physical or digital objects, or a combination.Your experience could either scale down its demands as things get hectic, or warn the user and ask at what level of intensity they’d like to continue.",
        "Respect users' environments and limit cognitive load"
    ],
    "Spatial Anchors": [
        "Create spatial anchors where users place them",
        "Release local spatial anchors you no longer need",
        "Avoid creating a grid of spatial anchors",
        "Always render anchored holograms within 3 meters of their anchor",
        "Group holograms that should form a rigid cluster",
        "Render highly dynamic holograms using the stationary frame of reference instead of a local spatial anchor"
    ],
    "Spatial mapping": [
        "Each application that uses spatial mapping should consider providing a 'scanning experience'; the process through which the application guides the user to scan surfaces that are necessary for the application to function correctly. (Microsoft p.139)",
        "Users do not know what makes a good scan. They need to be shown or told what to look for if they’re asked to evaluate a scan – flatness, distance from actual walls, etc. The developer should implement a feedback loop that includes refreshing the spatial mapping data during the scanning or exploration phase. In many cases, it may be best to tell the user what they need to do (e.g. look at the ceiling, look behind furniture), in order to get the necessary scan quality. (Microsoft p.117)",
        "Ensure the most essential elements fit in the smallest projected space, then progressively add elements as space allows. If an initial scan can’t find enough space, warn your user, and display guidance to help them meet the requirements."
    ],
    "Surface Detection": [
        "Low lighting and reflective surfaces. These can hinder the camera's ability to detect a surface. Let users know when this occurs and encourage them to move into a better lit space or textured surface if possible.",
        "By visualizing surfaces, the application can share with the user its understanding of the environment. For example, a holographic board game could visualize the horizontal surfaces that it has identified as 'tables', so the user knows where they should go to interact. (Microsoft p.134)",
        "Show people when to locate a surface and place an object. You can use the systemprovided coaching view to help people find a horizontal or vertical flat surface on which to place an object. After ARKit detects a surface, your app can display a custom visual indicator to show when object placement is possible. You can help people understand how the placed object will look in the environment by aligning your indicator with the plane of the detected surface.",
        "Use surface visualization to help users with object placement. Highlight surfaces that have been detected and are ready for object placement",
        "Use progressive disclosure. Let users know about changes in system status in a timely and relevant way. Using visual highlighting or displaying text to help communicate that a surface has been successfully detected",
        "As users move their phones, give instant feedback to show they’ve successfully detected a surface",
        "When a user successfully finds a surface, tell them what to do next. You can give them an object to place, encourage them to explore the space, or something else"
    ],
    "System Architecture": [
        "Let the technology do the hard work. Consider layering the experience so time-consuming activities happen in the background and more engaging activities happen in the foreground. More information on how to create experiences for a range of devices can be found in Spark AR technical documentation."
    ],
    "Text / Font": [
        "Text size is tricky in spatial computing. Typical users find 28px at 1.5 m comfortable. Avoid text smaller than 16px at 1.5 m distance.",
        "Keep text labels simple and readable.",
        "White text is preferred for visibility.",
        "When in doubt, make fonts larger and place them further away. If you use lighter font weights, increase the text size to maintain legibility.",
        "The minimum size of the LominoUI system font in the regular weight for comfortable reading is 19px-22px at 1.5m.",
        "Type locked to headpose is harder to read than type locked to space.",
        "Avoid serif fonts",
        "Make important text readable. Use screen space to display text used for critical labels, annotations, and instructions. If you need to display text in 3D space, make sure the text faces people and that you use the same type size regardless of the distance between the text and the labeled object.",
        "Use semibold or bold font weight HoloLens allows your experience to show beautiful high-resolution text. However, it is recommended that you avoid thin font weights such as light or semi-light because the vertical strokes can jitter in small font size. (Microsoft p.153)",
        "In general, we recommend testing specific applications for legibility and making font sizes as large as possible for a comfortable experience.(Microsoft p.101)",
        "For the near interaction at 0.45m(45cm), the minimum legible font's viewing angle and the height are 0.4°-0.5° / 3.14–3.9mm.(Microsoft p.224)",
        "For far interaction (2 meters): It depends on the characteristics of each font, but in general the recommended minimum viewing angle and the font height for legibility are around 0.35°-0.4°/ 12.21-13.97mm based on our user research studies. (Microsoft p.224)"
    ],
    "Textures - Visual Realism and Appearance of Objects": [
        "Strive for convincing illusions when placing realistic objects. Design detailed 3D assets with lifelike textures to create objects that appear to inhabit the physical environment in which you place them.",
        "Textures will appear more lifelike when you add visual noise. Repeating patterns and flat colors can seem fake to the human eye",
        "Add bumps, rips, variations, pattern disruptions, and other forms of visual noise. They break up repetitive tiling, and they make your assets look detailed and believable",
        "To make your assets look more believable, use Physically Based Rendering (PBR). PBR lets you add realistic detail to objects in your scene. It combines multiple textures to create an illusion of depth and texture, and it uses the physical behavior of light to give an object a natural feeling. These textures give definition to your object and enhance the realistic appearance of your AR world",
        "Normal mapping can add details to an object’s appearance. It makes the object’s texture look more realistic without having to add additional geometry. Normal maps are ideal for giving objects a photorealistic appearance. It’s a level of extra detail that you can apply to objects if you want",
        "A shadow plane is a virtual surface that sits beneath your object. The plane only contains shadows, not the objects themselves. Shadow planes are a simple way to emphasize the presence of objects in your scene. Include shadow plane geometry and a baked shadow plane texture with every asset",
        "Ambient occlusion is a rendering technique that controls the amount of shadow that falls on the surface of any object. When you use an ambient occlusion texture on an object, it looks more detailed and realistic"
    ],
    "Transition into AR/VR": [
        "Try to give the user a seamless transition to AR. Let the user launch the transition from a 2D interface to AR. It’s less jarring when the user is in control.",
        "Use an easy transition, like an animation or a fade out.",
        "Make a clear transition into AR. Use visuals to let users know they’re about to transition from a 2D screen into AR. You can dim the phone display or use effects to blur the screen when a transition is about to take place.",
        "When introducing users to a new scene, position the user at a strategic location and guide their attention towards objects of interest.",
        "Design an initial entry space or state for your experience that provides clear auditory and visual cues, guiding the user to an optimal position and direction.",
        "Upon placing users in a new virtual environment, give them 5 - 15 seconds to look around and orient themselves before prompting with further direction."
    ],
    "User Control & Freedom": [
        "Allow the user to reset the experience if they need to. Make the reset process quick and easy, and let users dive right back into the experience once it’s taken care of.",
        "Let people reset the experience if it doesn’t meet their expectations. Don't force people to wait for conditions to improve or struggle with object placement. Give them a way to start over again and see if they have better results."
    ],
    "Users' physical safety": [
        "Remind users to be aware of their surroundings. When movement is required, prompt people to check their environment so they avoid hurting themselves or causing damage. Their focus needs to be on what’s around them as well as their AR experience.",
        "Be mindful of people's safety. When people are immersed in an AR experience, they're not necessarily aware of their physical surroundings, so making rapid, sweeping, or expansive motions might be dangerous. Consider ways of making your app safe to operate; for example, a game could avoid encouraging large or sudden movements.",
        "Don’t make users walk backward The danger of bumping into furniture, small animals, or other objects is much greater when a user is moving backward",
        "Help users move safely in their space. If people are expected to move during the experience, remind them to make space before they have to make the movement.",
        "Keep the user safe. Sometimes, users can get too immersed in an AR experience. When they pay attention to the phone’s camera and ignore the real world, users can bump into objects, people, and might not notice hazards around them. Think about where your app is directing the user’s attention. Build in reminders to look around, and sporadically remind them to check their surroundings",
        "Provide firm but friendly reminders for users to be mindful of their environment. Remember the user will be mainly focused on what is happening on the screen rather than in the real world, and they will be moving around their space more than they would in a non-AR experience.",
        "Help users to be aware of their real world environment. Most AR experiences require people to move around their environment using the device screen to guide them.",
        "You can spatialize audio in your application to: Help users maintain awareness of their realworld surroundings. ",
        "Anticipate your user’s movement in their space. Content at or above eye level can be challenging or dangerous.",
        "Avoid directing the user to back up, especially if a forward headpose is necessary to the experience.",
        "Experiences must never put users at risk. For example, discourage or block interaction when driving.",
        "While users are moving, don’t include content in the air. Navigating cluttered environments can be challenging or dangerous if users are looking up."
    ],
    "Voice Commands": [
        "Maintain voice command consistency - If Go back goes to the previous page, maintain this behavior in your applications.",
        "Avoid using system commands - The following voice commands are reserved for the system. These should not be used by applications. Hey Cortana Select Go to start.",
        "Use concise commands - When possible, choose keywords of two or more syllables. Onesyllable words tend to use different vowel sounds when spoken by persons of different accents. Example: Play video is better than Play the currently selected video ",
        "Avoid similar sounding commands - Avoid registering multiple speech commands that sound very similar. Example: Show more and Show store can be very similar sounding.",
        "Use simple vocabulary - Example: Show note is better than Show placard",
        "Make sure commands are non destructive - Make sure any action that can be taken by a speech command is non destructive and can easily be undone in case another person speaking near the user accidentally triggers a command. ",
        "Unregister your app when not it use - When your app is not in a state in which a particular speech command is valid, consider unregistering it so that other commands are not confused for that one. ",
        "Test with different accents - Test your app with users of different accents.",
        "Voice Dictation can ease small and mediumsized text entry tasks.",
        "Users will need a clear indication of how to invoke voice, as well as a way to learn what voice interactions are available and possible.",
        "Use clear audio and visual feedback to show that the system is listening, the user is talking, and listening has ended."
    ],
    "appropriate interplay of virtual content and physical environments": [
        "In any AR experience, try to blend your AR world into the user’s physical environment. Whether the goal is realism or wackiness, your AR objects should engage with and reflect their environment. Use shadows, lighting, occlusion, reflection, and collision to help your objects take up space in the real world",
        "Design your virtual objects to live in the real world. In an AR experience, each object should feel like it’s located in the same scene as every other object. Simulate the presence of objects by using shadows, lighting, ambient occlusion, physics, and reflection. Just like real objects, AR objects should react to the realworld environment",
        "The goal is to keep users in their world, so reality should be a major player in your creation.",
        "Design UI components for immersive experiences that aim to visually blend real & virtual spaces. Create a visually clean UI that lends itself seamlessly to the immersive experience you’re building.",
        "Using information from ARKit, you can scale objects properly and position them on detected realworld surfaces, reflect environmental lighting conditions and simulate camera grain, cast top-down diffuse object shadows on real-world surfaces, and update visuals as the camera's position changes.",
        "Visual Simplicity. Less is more. The fewer pixels you use, the more magical each of those pixels will seem. You want to keep the user in their world, so reality should be a major player in your experience.",
        "Consider how virtual objects with reflective surfaces show the environment. Reflections in ARKit are approximations based on the environment captured by the camera. To help maintain the illusion that an AR experience is real, prefer small or coarse reflective surfaces that downplay the effect of these approximations.",
        "Optimize lighting conditions in your scene. To let your virtual objects blend in with the real world, make sure they have the right lighting. There’s a light estimation system built right into ARCore. It’ll let you render lighting conditions that adjust dynamically to the surrounding area",
        "Don't occlude reality or add overwhelming distractions.",
        "Also note that the more rules of reality you follow, the more rooted in reality your experience will seem, and the more grounded your user will remain."
    ],
    "content spawn mechanic": [
        "Minimize surprises. The nature of the medium lets objects appear from nowhere. Unless you’re intentionally startling the user for effect, avoid jump scares and popouts. Prepare users before introducing a new object or present it gently, allowing it to draw their attention.",
        "Use natural entries like arising from a surface, coming from behind real world objects, or entering through a window. Avoid teleporting in or ghosting through walls. To let users feel the reality of your pixels, follow the rules of the real world whenever possible.",
        "Spawn objects in accessible areas, not inside physical objects, and ideally within 10 feet of the user. If an object spawns in an unmeshed piece of black furniture, the user can’t interact with it. For interactions that are important for the experience, provide a clear backup option."
    ],
    "depth perception - Visual Realism and Appearance of Objects": [
        "Create objects that move from one part of the scene to another. Think of a frog that leaps from the floor up to a table, or a train that rides from the background to the foreground. It’ll make your experience feel less like a store window and more like a world, and, as a benefit, the scene will feel inhabited and alive. Leverage foreground & background to communicate depth",
        "Use visual techniques to give depth and distance to your scene. In an AR experience, users can have trouble identifying how far away, and how deep into the scene, of virtual objects. Flesh out your scene by creating the effect of depth. Use techniques like shadow planes, occlusion, and texture. You can also play with perspectives, like placing large objects close to the user and smaller ones far away",
        "Design your scene so it’s easy for users to perceive the depth and distance of objects"
    ],
    "gaze (input)": [
        "To be easily accessible, headgaze and dwell targets need to be large enough to comfortably look at, and hold one's head stable on the target for the prescribed time. We recommend a minimum target size of 2 degrees to achieve the most comfortable experience. (Microsoft p.211)",
        "Seek Goldilocks speed - Dwell interactions can have different timers based on impact of navigation - more frequently used functions will generally benefit from faster fill times, while more consequential functions may benefit from longer fill times. When using a fill-effect to show these timers, animation curves of the fill color can positively influence a feeling of faster fill times. Consideration should be taken to enable user decision from fast/medium/slow fill speed overrides. (Microsoft p.208)",
        "Onset delay: When the user starts looking at a target, nothing should immediately happen as this may result in an unpleasant and overwhelming user experience. Instead start a timer to detect whether the user is intentionally staring at the target or merely glancing over it. We recommend an onset time of 150-250 ms in a given proximity (meaning the user is fixating vs. looking around on a large target)(Microsoft p.208)",
        "Use eye gaze as a secondary input to supplement or clarify other input methods. For example, with two objects in range, you might use eye gaze to determine the target",
        "Be wary of using eye gaze alone as a targeting method. Targeting with eye tracking alone can feel unnatural and fatiguing to users because it requires they consciously control their eye movement, risking cognitive overload.",
        "High frequency buttons should be large, easier to hit with head-gaze Stay near eye height to avoid ergonomic straining (Microsoft p.211)",
        "Prevent flickering with delayed feedback: Use a short delay before starting visual feedback to avoid flickering when someone passes over a dwell target. For buttons interacted with frequently, keep the delay very short so the application feels reactive. For buttons that are interacted with infrequently, a longer delay can be appropriate to avoid the interface feeling twitchy. (Microsoft p.211)",
        "Finish: If the user kept fixating the target (for another 650-850 ms), complete the dwell activation and select the looked-at target.(Microsoft p.208)",
        "Headpose targeting is easiest when content is at eye-level.",
        "Only use eye gaze targeting for special experiences like interactions designed to address accessibility or mobility limitations, or as an intentional challenging ingame mechanic.",
        "Strive for subtle blended hover feedback: What seems great visual feedback for head gaze can result in terrible, overwhelming experiences using eye gaze. Quick sudden highlight changes (on/off) may result in flickery feedback when looking around. So, when providing hover feedback, we recommend using a smoothly blended-in highlight (and blended-out when looking away). Over the course of 500-1000 ms the highlight would increase in intensity. In addition, we also recommend using a blend-out when fading out the hover feedback. Research has shown that quick motions and contrast changes are very noticeable in your peripheral vision. The fade-out doesn't have to be as slow as the blend-in. (Microsoft p.224)",
        "Start dwell feedback: After ensuring that the user is intentionally looking at the target, start showing dwell feedback to inform the user that the dwell activation is being initiated.(Microsoft p.208)",
        "Continuous Feedback: While the user keeps looking at the target, show a continuous progress indicator so that the user knows that they have to keep looking at the target. In particular for eye-gaze input, we recommend pulling in the user's visual attention by starting out with a bigger circle or sphere that contracts into a smaller version. Showing an indicator for the final state (small circle) helps to communicate to the user when the dwell will be finished.(Microsoft p.208)",
        "Try to keep Low frequency buttons out of the way of frequent head-gaze paths to avoid accidental activation (Microsoft p.211)",
        "Avoid enforcing unnatural eye movements as input: Do not force users to perform specific eye movements (gaze gestures) to trigger actions in your application. (Microsoft p.224)"
    ],
    "hand &finger gestures": [
        "Ensure controls and gestures are ambidextrous.",
        "More hand postures create more room for misinterpretation. Limiting your app’s interaction design to a few hand postures will both improve system performance and feel more accurate.",
        "Design two-handed interactions with dominant and nondominant handedness in mind. Combine Control interactions with secondary gesture interactions.",
        "Stick to simple gestures and make them obvious. Users may not have experience with AR and may also have no awareness that they are entering an experience with it. As a result, we have to make sure gestures are intuitive and touch interactions are clear.",
        "Two-handed gestures should be used sparingly. For example, holding a mobile device in one hand and performing a gesture with the other hand can quickly become tiring.",
        "Where gestures are less familiar, both illustration and text is preferable.",
        "Be wary of potentially conflicting gestures. A twofinger pinch gesture, for example, is similar to a twofinger rotation gesture. If you implement two similar gestures like this, be sure to test your app and make sure they're interpreted properly."
    ],
    "headlocked content": [
        "In mixed reality experiences, HUDs have the potential to cause significant discomfort and must be adapted to the more immersive context. Specifically, HUDs that are rigidly locked to the user’s head orientation are likely to produce discomfort. If an app requires a HUD, we recommend body locking rather than head locking.(Microsoft p.100)",
        "Implementing 1:1 HUD rotation and translation relative to the user’s head motions should always be avoided",
        "We strongly discourage rendering head-locked content, which stays at a fixed spot in the display (such as a HUD). In general, head-locked content is uncomfortable for users and does not feel like a natural part of their world. (Microsoft p.107)",
        "Head-locked content should usually be replaced with holograms that are attached to the user or placed in the world itself. For example, cursors should generally be pushed out into the world, scaling naturally to reflect the position and distance of the object under the user's gaze. (Microsoft p.107)",
        "Smart Content: Make objects and characters aware of the field of view boundaries and subtly shift to stay within them.",
        "When content needs immediate attention and focus, the lazy follow mechanic can bring content gracefully back into the field of view.",
        "Do not use headlocked content that occludes most of the field of view",
        "Avoid headlocking large text objects. We naturally turn to read text. Attaching text to head position creates an awkward reading experience.",
        "Inertia lessens the discomfort of headlocked content."
    ],
    "interactive & reactive environment / content": [
        "Explore even more engaging methods of interaction. Gestures aren't the only way for people to interact with virtual objects in AR. Your app can use other factors, like motion and proximity, to bring content to life. A game character, for example, could turn its head to look at a person as they walk toward it.",
        "Scale information in the scene as the user moves closer. For example, if the objects are further away in the scene than the information associated with the object should be “glanceable” (i.e. reduced or minimized) whereas information can become more detailed and readable if it is closer to the viewer.",
        "The category of implicit actions closely relates to user intent. The idea is that holograms or user interface elements react in an instinctual way that may not even feel like the user is interacting with the system at all, but rather that the system and the user are in sync. (Microsoft p.111)",
        "Attentive holograms: Holograms that subtly react when being gazed upon. This can range from slightly glowing UI elements, a slowly blooming flower to a virtual dog starting to look back at the user and wagging its tail. This interaction might provide an interesting sense of connectivity and satisfaction in your application. (Microsoft p.111)",
        "Billboarding Certain objects–such as things that are planar–make sense to continuously face the user (billboard) during the movement sequence to encourage ease of access and legibility.",
        "Remember that your user is wearing a device that can read where they're looking. You can either lead them to the content or move the content into their FOV. If it makes sense, skip the locator and let the content come to the user."
    ],
    "notifications": [
        "If you ask users to read something, consider attaching the notification to an object or keep it stationary in space.",
        "Use notifications sparingly. Poor notification design severely disrupts users’ experience.",
        "When in doubt, make the notification larger and a little more distant. Take care with occlusion, especially when the notification is essential."
    ],
    "privacy": [
        "Don’t request sensitive information such as credit cards, social security numbers, or other personal data by voice only. Never deliver potentially sensitive information via audio feedback.",
        "Respect users' privacy. Be clear about their choices regarding their data. Put them at ease by giving them control over their information and privacy settings."
    ],
    "subtiltles": [
        "Consider accessibilityfriendly typefaces such as the Dyslexie font, designed while taking the different characteristics of dyslexia into consideration.",
        "Support adjustable font size",
        "Adding relevant nondialogue information such as environmental noises, music and sound effects and nonverbal character sounds.",
        "Exploring placement and frame of reference for subtitles (relative to the user, are they in world, are they attached to an on screen speaker, etc.).",
        "Using clear, high contrast text on an additive display"
    ],
    "visual cues for object manipulation": [
        "When both surfaces are virtual, make sure there’s a visual distinction between the two. Identical surfaces can blur together in AR.",
        "Visually highlight only the surface that the user is viewing or pointing at. Avoid highlighting multiple surfaces at once, and create visual distinctions between different planes",
        "Add a boundary to show users how far they can move an object. It can stop users from translating the object so far away that it becomes impossible to view or manipulate.",
        "Moving objects in AR can feel less realistic then simply looking at them. Make it easy for the user to move objects. And, when the object is being moved, highlight the surfaces where the object can be placed.",
        "Guide users as they place objects in the scene. Help them to avoid placing objects at an uncomfortable distance",
        "Use visual indicators to guide users as they place objects. Shadows can help indicate the destination point and make it easier for users to understand where an object will be placed on a detected surface",
        "It's important to show people how the virtual objects relate to the real world. For example, effects such as shadows under a virtual object on a detected plane help people to quickly understand where an object will land if they decide to place it in the scene.",
        "Explore new methods for manipulation. Handles, turntables, and spinners, for example, provide familiar visual queues"
    ],
}


# Step 1: Embeddings nur für Guidelines
guideline_embeddings = model.encode(df["Guideline"].tolist(), convert_to_tensor=True)

# Step 2: Embeddings für ARcis Topics
topic_embeddings = {
    topic: model.encode(examples, convert_to_tensor=True)
    for topic, examples in topics.items()
}

# Step 3: Berechne Similarity Scores
topic_scores = []
assigned_topics = []
all_scores = {topic: [] for topic in topics}

for emb in guideline_embeddings:
    avg_similarities = {}
    for topic, topic_embs in topic_embeddings.items():
        sims = util.cos_sim(emb, topic_embs)
        avg_sim = sims.mean().item()
        avg_similarities[topic] = avg_sim
        all_scores[topic].append(round(avg_sim, 4))

    best_topic = max(avg_similarities, key=avg_similarities.get)
    assigned_topics.append(best_topic)
    topic_scores.append(round(avg_similarities[best_topic], 4))

# Ergebnisse zur Original-DataFrame hinzufügen
df["Assigned Topic"] = assigned_topics
df["Similarity Score"] = topic_scores
df["Topic + Score"] = df["Assigned Topic"] + " (" + df["Similarity Score"].astype(str) + ")"

print(df)

# Einzelne Scores pro Topic
for topic in topics:
    df[f"Score: {topic}"] = all_scores[topic]


# Export der vollständigen Datei
output_file_full = "/AR_DesignGuidelines_STS_analyzed.xlsx"
df.to_excel(output_file_full, index=False)

